# CFT Latency Example

## Setup and Helper Functions

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

from fau_colors import cmaps, register_fausans_font

import biopsykit as bp
from biopsykit.protocols import CFT

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

register_fausans_font()
palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="talk", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
# get path to analysis results
data_path = Path("../../2022_scientific_reports/data")
base_path = Path("..")

In [ ]:
results_path = base_path.joinpath("results")
plot_path = results_path.joinpath("plots")
bp.utils.file_handling.mkdirs([results_path, plot_path])

## Create CFT Object

The [CFT](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.protocols.cft.html#biopsykit.protocols.cft.CFT) is a protocol and thus part of the Protocol API of `BioPsyKit`.

In [ ]:
cft = CFT()
cft

## Load Data

In [ ]:
hr_dict = bp.example_data.get_mist_hr_example()

# Alternatively: Use your own data
# hr_dict = bp.io.ecg.load_hr_phase_dict("<path-to-heart-rate-data-dictionary>")
df_hr = hr_dict["MIST3"]

df_hr.head()

## Compute CFT Parameter

Extract CFT Interval, i.e., the interval of the data where the Cold Face stimulus was actually applied (corresponds to the duration of the "CFT" phase specified in the [CFT](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.protocols.cft.html#biopsykit.protocols.cft.CFT) object) and compute baseline heart rate (the mean heart rate during the "Baseline" phase specified in the [CFT](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.protocols.cft.html#biopsykit.protocols.cft.CFT) object).

Note: See [CFT.extract_cft_interval()](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.protocols.cft.html#biopsykit.protocols.cft.CFT.extract_cft_interval) and [CFT.compute_cft_parameter()](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.protocols.cft.html#biopsykit.protocols.cft.CFT.compute_cft_parameter) for further information!

In [ ]:
df_cft = cft.extract_cft_interval(df_hr)
bl = cft.baseline_hr(df_hr)

print("Baseline Heart Rate: {:.2f}".format(bl))
df_cft.head()

In [ ]:
cft.compute_cft_parameter(df_hr)

## Plot CFT Data

In [ ]:
from fau_colors import colors_all

def _cft_plot_add_param_annotations(cft, data, cft_params, times_dict, ax, bbox, **kwargs):
    if kwargs.get("plot_baseline", True):
        _cft_plot_add_baseline(cft, cft_params, times_dict, ax)
    if kwargs.get("plot_mean", True):
        _cft_plot_add_mean_bradycardia(cft, data, cft_params, times_dict, ax, bbox)
    if kwargs.get("plot_onset", True):
        _cft_plot_add_onset(cft, data, cft_params, times_dict, ax, bbox)
    if kwargs.get("plot_peak_brady", True):
        _cft_plot_add_peak_bradycardia(cft, data, cft_params, times_dict, ax, bbox)

def _cft_plot_add_peak_bradycardia(
    cft,
    data: pd.DataFrame,
    cft_params,
    cft_times,
    ax: plt.Axes,
    bbox,
) -> None:

    color_key = "fau"
    if isinstance(data.index, pd.DatetimeIndex):
        brady_loc = cft_params["peak_brady"]
        brady_x = brady_loc
        brady_y = float(data.loc[brady_loc])
    else:
        brady_loc = cft_params["cft_start_idx"] + cft_params["peak_brady_idx"]
        brady_x = data.index[brady_loc]
        brady_y = float(data.iloc[brady_loc])

    hr_baseline = cft_params["baseline_hr"]
    max_hr_cft = float(cft.extract_cft_interval(data).max())
    cft_start = cft_times["cft_start"]

    color = getattr(colors_all, color_key)
    color_adjust = getattr(colors_all, f"{color_key}_dark")

    # Peak Bradycardia vline
    ax.axvline(x=brady_x, ls="--", lw=2, alpha=0.6, color=color)

    # Peak Bradycardia marker
    ax.plot(
        brady_x,
        brady_y,
        color=color,
        marker="o",
        markersize=7,
    )

    # Peak Bradycardia hline
    xmax = brady_x + 20

    ax.hlines(
        y=brady_y,
        xmin=brady_x,
        xmax=xmax,
        ls="--",
        lw=2,
        color=color_adjust,
        alpha=0.6,
    )


    brady_x_offset = brady_x + 5

    ax.annotate(
        "",
        xy=(brady_x_offset, brady_y),
        xytext=(brady_x_offset, hr_baseline),
        arrowprops={
            "arrowstyle": "<->",
            "lw": 2,
            "color": color_adjust,
            "shrinkA": 0.0,
            "shrinkB": 0.0,
        },
    )

    # Peak Bradycardia Text
    ax.annotate(
        "Peak Brady.",
        xy=(brady_x_offset, brady_y - 2),
        xytext=(10, -5),
        textcoords="offset points",
        bbox=bbox,
        ha="left",
        va="top",
    )

    # Peak Bradycardia Latency arrow
    ax.annotate(
        "",
        xy=(cft_start, max_hr_cft),
        xytext=(brady_x, max_hr_cft),
        arrowprops={
            "arrowstyle": "<->",
            "lw": 2,
            "color": color,
            "shrinkA": 0.0,
            "shrinkB": 0.0,
        },
    )

    # Peak Bradycardia Latency Text
    ax.annotate(
        "CFT Latency",
        xy=(brady_x, max_hr_cft),
        xytext=(-7.5, 10),
        textcoords="offset points",
        bbox=bbox,
        ha="right",
        va="bottom",
    )

def _cft_plot_add_baseline(
    cft,
    cft_params,
    cft_times,
    ax: plt.Axes,
) -> None:
    color_key = "tech"

    # Baseline HR
    ax.hlines(
        y=cft_params["baseline_hr"],
        xmin=cft_times["plot_start"],
        xmax=cft_times["cft_end"],
        ls="--",
        lw=2,
        color=getattr(colors_all, color_key),
        alpha=0.6,
    )

def _cft_plot_add_mean_bradycardia(
    cft,
    data: pd.DataFrame,
    cft_params,
    cft_times,
    ax: plt.Axes,
    bbox,
) -> None:

    color_key = "nat"
    mean_hr = cft_params["mean_hr_bpm"]
    cft_start = cft_times["cft_start"]
    cft_end = cft_times["cft_end"]

    # Mean HR during CFT
    ax.hlines(
        y=mean_hr,
        xmin=cft_start,
        xmax=cft_end,
        ls="--",
        lw=2,
        color=getattr(colors_all, color_key),
        alpha=0.6,
    )

    x_offset = cft_end - 10

    # Mean Bradycardia arrow
    ax.annotate(
        "",
        xy=(x_offset, mean_hr),
        xytext=(x_offset, cft_params["baseline_hr"]),
        arrowprops={
            "arrowstyle": "<->",
            "lw": 2,
            "color": getattr(colors_all, color_key),
            "shrinkA": 0.0,
            "shrinkB": 0.0,
        },
    )

    # Mean Bradycardia Text
    ax.annotate(
        "Mean Brady.",
        xy=(x_offset, mean_hr - 2),
        xytext=(0, -5),
        textcoords="offset points",
        bbox=bbox,
        ha="left",
        va="top",
    )

def _cft_plot_add_onset(
    cft,
    data: pd.DataFrame,
    cft_params,
    cft_times,
    ax: plt.Axes,
    bbox,
) -> None:

    color_key = "med"
    color = getattr(colors_all, color_key)

    if isinstance(data.index, pd.DatetimeIndex):
        onset_idx = cft_params["onset"]
        onset_x = onset_idx
        onset_y = float(data.loc[onset_idx])
    else:
        onset_idx = cft_params["cft_start_idx"] + cft_params["onset_idx"]
        onset_y = float(data.iloc[onset_idx])
        onset_x = data.index[onset_idx]

    # CFT Onset vline
    ax.axvline(onset_x, ls="--", lw=2, alpha=0.6, color=color)

    # CFT Onset marker
    ax.plot(
        onset_x,
        onset_y,
        color=color,
        marker="o",
        markersize=7,
    )

    # CFT Onset arrow
    ax.annotate(
        "",
        xy=(onset_x, onset_y),
        xytext=(cft_times["cft_start"], onset_y),
        arrowprops={
            "arrowstyle": "<->",
            "lw": 2,
            "color": color,
            "shrinkA": 0.0,
            "shrinkB": 0.0,
        },
    )

    # CFT Onset Text
    ax.annotate(
        "CFT Onset",
        xy=(onset_x, onset_y),
        xytext=(-10, -10),
        textcoords="offset points",
        bbox=bbox,
        ha="right",
        va="top",
    )

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
time_baseline = 60
time_recovery = 60


data = df_hr.copy()

cft_params = cft.compute_cft_parameter(data, return_dict=True)
data.index = (data.index - data.index[0]).view(np.int64) / 1e9

times_dict = cft._cft_plot_get_cft_times(data, time_baseline, time_recovery)
df_plot = cft._cft_plot_extract_plot_interval(data, times_dict)

bbox = {
    "fc": (1, 1, 1, plt.rcParams["legend.framealpha"]),
    "ec": plt.rcParams["legend.edgecolor"],
    "boxstyle": "round",
}

bp.signals.ecg.plotting.hr_plot(heart_rate=df_plot, ax=ax, plot_mean=False)
cft._cft_plot_add_phase_annotations(ax, times_dict)
_cft_plot_add_param_annotations(cft, data, cft_params, times_dict, ax, bbox)
cft._cft_plot_style_axis(data, ax)

fig.tight_layout()
fig.savefig(plot_path.joinpath("img_cft_example.pdf"), transparent=True)